## RNN
Рекуррентные нейронные сети (Recurrent neural network; RNN) — вид нейронных сетей, где связи между элементами образуют направленную последовательность. 

Благодаря этому появляется возможность обрабатывать серии событий во времени или последовательные пространственные цепочки. 

Рекуррентные сети могут использовать свою внутреннюю память для обработки последовательностей произвольной длины. 

![RNN](https://cdn-images-1.medium.com/max/800/1*DItCSHJ-NA2wy4lFfXv-_Q.png)

## LSTM

LSTM - одна из разновдностей RNN сетей, разработанная для предотвращения исчезающих градиентов. 

И благодаря своей архитектуре и внутренней памяти позволяющая делать хорошие предсказания на временных рядах.

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
%matplotlib inline


df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/air_visit_data.csv.zip') #пропишите путь до файла

In [0]:
# Из всего массива ресторанов выбираем тот, который мы проверяем разными методами
df = df[df['air_store_id'] == 'air_cb7467aed805e7fe']
df.drop('air_store_id', inplace=True, axis=1)
df.dropna(how='any', inplace=True)
df.head(5)

In [0]:
# Приводим колонку даты к типу данных - дата
df['visit_date'] = pd.to_datetime(df['visit_date'])

In [0]:
# Делим на трейн и тест
train = df[df['visit_date'] < datetime(2017, 1, 17)].reset_index(drop=True).drop('visit_date', axis=1)
test = df[df['visit_date'] >= datetime(2017, 1, 17)].reset_index(drop=True).drop('visit_date', axis=1)

In [0]:
train.head()

In [0]:
train.shape, test.shape

In [0]:
# RNN предпочтительно скалировать MinMax скейлером
from sklearn.preprocessing import MinMaxScaler
mc = MinMaxScaler(feature_range = (0, 1))
mc.fit(train)
train_scaled = mc.transform(train)

In [0]:
# Для LSTM надо задать специальную структуру данных
# n - длина исторических данных, на которых мы учимся
n = 30
X_train = []
y_train = []
# Каждая строка нового массива - это вектор длины n, ответом для которого является значение ряда в точке n+1
for i in range(n, len(train_scaled)):
    X_train.append(train_scaled[i-n:i, 0])
    y_train.append(train_scaled[i, 0])
    
# Для кераса и данной реализации данные должны иметь вид numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

# Так же надо добавить больше измерений в данные, отвечающих за кол-во признаков.
# У нас всего один признак, поэтому в третье измерение дописываем значение 1
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [0]:
# Подгружаем нужные библиотеки
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [0]:
# Инициализируем RNN
regressor = Sequential()

# Добавляем первый LSTM слой и Dropout регурялизацию
# Когда мы стакаем несколько LSTM слоев, то должны прописывать return_sequences = True, 
# чтобы на следующий слой так же передавался трехмерный array 
regressor.add(LSTM(units = 20, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Добавляем второй LSTM слой и Dropout регурялизацию
# Стаканье слоев помогает искать временные тренды и паттерны, что на практике дает хорошие результаты
regressor.add(LSTM(units = 20, return_sequences = True))
regressor.add(Dropout(0.2))

# Добавляем третий LSTM слой и Dropout регурялизацию
regressor.add(LSTM(units = 20, return_sequences = True))
regressor.add(Dropout(0.2))

# # Добавляем четвертый LSTM слой и Dropout регурялизацию
# regressor.add(LSTM(units = 20, return_sequences = True))
# regressor.add(Dropout(0.2))

# Добавляем пятый LSTM слой и Dropout регурялизацию
regressor.add(LSTM(units = 20))
regressor.add(Dropout(0.2))

# Добавляем выходящий слой
regressor.add(Dense(units = 1))

# Компиляция
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Обучение RNN на тренировочном датасете
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)


In [0]:
# Предсказание
# Чтобы получить ту же структуру данных для предсказания, что и в тренировочном сете, 
# для каждой точки теста должно быть n точек истории.
# Значит, мы должны объединить трейн и тест, чтобы получить эти точки.
df_full = pd.concat((train, test), axis = 0).reset_index(drop=True)
ttest = df_full[len(df_full) - len(test) - n:].values
ttest = ttest.reshape(-1,1)
# Делаем скалирование
ttest = mc.transform(ttest)
# Собираем датасет, как и в случае трейна
X_test = []
for i in range(n, n+test.shape[0]):
    X_test.append(ttest[i-n:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [0]:
# Делаем предсказание
preds = regressor.predict(X_test)
# Инвертируем предсказание обратно в обычные единицы измерения
preds = mc.inverse_transform(preds)

In [0]:
fig = plt.figure(figsize=(25, 5))
plt.plot(test, label='real')
plt.plot(preds, color='red', label='pred')
plt.legend()
plt.show()

In [0]:
from sklearn.metrics import mean_squared_error as score

In [0]:
score(test, preds)**(1/2)

Возможные улучшения:

*    Найти больше тренировочных данных (но у нас нет)
*    Увеличить количество используемой истории. Вместо 30 или 60 дней взять еще больше.
*    Добавить другие признаки. Другие рестораны или остальные признаки по ним. 
*    Добавить больше LSTM слоев.
*    Добавить больше нейронов в каждый LSTM слой. Не 50, а больше.


Ссылки для изучения:

1. [Обзор с хорошими ссылками в конце (обязательно их посмотрите!)](https://towardsdatascience.com/learn-how-recurrent-neural-networks-work-84e975feaaf7)
2. [Про LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
3. [Еще раз про LSTM на примере труб](https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714)